<a href="https://colab.research.google.com/github/mohamadrezanumberone/Deep-Compression-PyTorch/blob/master/Custom_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [ ]:
import torch
import torchvision
from torchvision.datasets import CIFAR10
from torchvision import transforms
import torch.nn as nn
import copy
import time
import torch.optim as optim
import numpy as np
import torchvision.transforms as transforms
from torchvision import datasets, models, transforms
from __future__ import print_function
from __future__ import division
import matplotlib.pyplot as plt
import torchvision.models as models
import torchsummary
from sklearn.cluster import KMeans
from skimage.io import imread
from skimage.color import rgb2gray
import numpy as np
%matplotlib inline
from scipy import ndimage
from PIL import Image
import torch
from collections import OrderedDict 

# Initialization


In [ ]:
batch_size = 256
num_class = 10
num_epochs = 24
feature_extract = True
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


# Dataset

In [ ]:
train_dataset = CIFAR10('./CIFAR10', train= True, transform= transforms.ToTensor(), target_transform = None, download= True)
test_dataset = CIFAR10('./CIFAR10', train= False, transform= transforms.ToTensor(), target_transform = None, download= True)


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./CIFAR10/cifar-10-python.tar.gz to ./CIFAR10
Files already downloaded and verified


In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
for batch_number, (images, labels) in enumerate(train_loader):
    print(batch_number, images.shape)

In [ ]:
one_train_batch_imgs, one_train_batch_lbls = next(iter(train_loader))
print(one_train_batch_imgs.shape,"\n", one_train_batch_lbls)

torch.Size([256, 3, 32, 32]) 
 tensor([1, 6, 2, 7, 9, 1, 7, 0, 9, 8, 4, 4, 3, 9, 1, 8, 8, 9, 1, 6, 4, 5, 7, 2,
        8, 1, 0, 6, 9, 0, 9, 7, 5, 8, 8, 9, 1, 1, 3, 7, 8, 2, 8, 5, 3, 5, 1, 9,
        1, 4, 6, 2, 6, 0, 8, 0, 7, 2, 3, 0, 3, 1, 9, 6, 3, 1, 2, 2, 9, 2, 4, 9,
        7, 1, 4, 9, 3, 9, 1, 9, 3, 7, 0, 3, 5, 7, 5, 8, 6, 1, 3, 0, 4, 9, 4, 6,
        6, 5, 1, 0, 8, 7, 2, 6, 8, 2, 6, 9, 2, 8, 5, 9, 3, 6, 5, 5, 0, 4, 4, 1,
        5, 4, 5, 0, 3, 5, 5, 1, 5, 3, 2, 8, 8, 7, 3, 6, 2, 7, 8, 2, 2, 4, 7, 2,
        7, 9, 8, 4, 8, 9, 3, 3, 1, 5, 5, 5, 2, 4, 3, 3, 1, 6, 9, 4, 9, 0, 9, 3,
        8, 7, 6, 0, 3, 6, 1, 9, 9, 5, 5, 1, 9, 8, 2, 6, 8, 2, 1, 0, 8, 0, 8, 5,
        0, 2, 0, 5, 9, 1, 1, 0, 8, 7, 8, 9, 1, 8, 6, 5, 9, 1, 4, 9, 7, 4, 8, 4,
        7, 9, 3, 4, 3, 0, 7, 4, 6, 0, 2, 4, 9, 9, 7, 7, 0, 5, 6, 9, 9, 7, 4, 3,
        1, 2, 8, 8, 1, 0, 9, 2, 6, 6, 1, 9, 0, 4, 6, 8])


In [ ]:
dataloaders = {'train': train_loader ,'test': test_loader}

# Mounting Google Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Copying Content

In [ ]:
!cp "/content/drive/My Drive/raw_imgs" "/content/"

In [ ]:
!cp "/content/drive/My Drive/cls_imgs" "/content/"

In [ ]:
!cp "/content/drive/My Drive/cnt_list" "/content/"

# Load

In [ ]:
imgs = torch.load('raw_imgs')

In [ ]:
clustered_imgs = torch.load('cls_imgs')

In [ ]:
cnt_ld = torch.load('cnt_list')

# Model

In [93]:
class ConvApprox(nn.Module):

    def __init__(self, num_class, in_channels, out_channels, kernel_size, stride):

        super(ConvApprox, self).__init__() 
        self.weight = torch.randn(out_channels, in_channels, kernel_size, kernel_size)
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        # unfold
        self.unfold = nn.Unfold(kernel_size=(kernel_size, kernel_size), stride=stride)
        

    def forward(self, input, center):
     
        img_unfold = self.unfold(input)
        print(img_unfold.shape)
     
        variance = img_unfold.var(dim=1, keepdim=True)
        same_ids = variance == 0
        same_data = []
        diff_data = []
        for i, s in enumerate(same_ids):
            same_data.append(img_unfold[i, :, s[0]])
            diff_data.append(img_unfold[i, :, torch.logical_not(s[0])])
    
        
        return same_data, diff_data

        


        



In [ ]:
data_batch1 = torch.tensor(clustered_imgs[0]).unsqueeze(0).unsqueeze(0).float()
data_batch2 = torch.tensor(imgs[0]).float()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [51]:
a = torch.cat((torch.linspace(1, 10, 5).repeat(3, 1), torch.linspace(1, 5, 5).repeat(3, 1)), dim=1)
b = a.view(2, 3, 5)
a

tensor([[ 1.0000,  3.2500,  5.5000,  7.7500, 10.0000,  1.0000,  2.0000,  3.0000,
          4.0000,  5.0000],
        [ 1.0000,  3.2500,  5.5000,  7.7500, 10.0000,  1.0000,  2.0000,  3.0000,
          4.0000,  5.0000],
        [ 1.0000,  3.2500,  5.5000,  7.7500, 10.0000,  1.0000,  2.0000,  3.0000,
          4.0000,  5.0000]])

In [45]:
torch.linspace(1, 10, 5).repeat(3 ,1)

tensor([[ 1.0000,  3.2500,  5.5000,  7.7500, 10.0000],
        [ 1.0000,  3.2500,  5.5000,  7.7500, 10.0000],
        [ 1.0000,  3.2500,  5.5000,  7.7500, 10.0000]])

In [100]:
out_channels, in_channels, kernel_size, stride, num_clusters = 32, 3, 3, 1, 3
weight = torch.randn(out_channels, in_channels, kernel_size, kernel_size)
unfold = nn.Unfold(kernel_size=(kernel_size, kernel_size), stride=stride)


# Clustered image input
input1 = data_batch1.clone()
clusterimg_unfold = unfold(input1)

# Image input
input2 = data_batch2.clone()
img_unfold = unfold(input2)
b, c, n = img_unfold.shape

img_unfold = img_unfold.view(b, 3, kernel_size**2, n)
num_batch, _, num_windows = clusterimg_unfold.size()

# Mean 
avg_unfold = torch.mean(img_unfold, dim=2)

variance = clusterimg_unfold.var(dim=1, keepdim=True)
same_ids = variance == 0

same_data = []
diff_data = []

output = torch.zeros((num_batch, 1, out_channels, num_windows))

w = weight.view(out_channels, in_channels, -1)

for i, s in enumerate(same_ids):
    # Same
    sames = clusterimg_unfold[i, 0, s[0]]
    # Diff
    diffs = clusterimg_unfold[i, :, torch.logical_not(s[0])].t().flatten()
    #
    same_data = torch.matmul(torch.sum(w, dim=-1), avg_unfold[i, :, s[0]])
    diff_data = torch.matmul((w).sum(-1) , avg_unfold[i, : ,torch.logical_not(s[0])])
    # 
    output[i, 0, :, s[0]] += same_data
    output[i, 0, :, torch.logical_not(s[0])] += diff_data.clone()

#Fold 
fold = nn.Fold(output_size=(int(np.sqrt(num_windows)), int(np.sqrt(num_windows))), kernel_size=(1, 1))
output = fold(output.squeeze(0))

# output.requires_grad = True 
print(diff_data[0].shape)

torch.Size([556])


In [101]:
output.shape

torch.Size([1, 32, 30, 30])

In [ ]:
model = ConvApprox(num_class, 3, 32, 3, 1)
output = model(data_batch, centers_batch)

torch.Size([3, 9, 900])


In [ ]:
[print(o.shape) for o in output[0]]
[print(o.shape) for o in output[1]]

In [ ]:
sample_filter = torch.randn(1, 3, 3, 3)
sample_filter.shape[2]

3

In [ ]:
model(torch.randn(10, 3, 32, 32).to(device)).shape

torch.Size([10, 10])

In [ ]:
a = next(iter(dataloaders['train']))
a[0].shape
# model(a[0].cuda()).shape

torch.Size([256, 3, 32, 32])

# Config

In [ ]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Trainer

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'test':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best test Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [ ]:
model_ft, hist = train_model(model, dataloaders, criterion, optimizer, num_epochs=num_epochs)